In [ ]:
# Qiskit packages
import qiskit, qiskit_aer#, qiskit_ibm_runtime, qiskit_ibm_catalog, qiskit_serverless

from python_files import Query as qry, RS_prep as rsp, General as grl, Stabilizer_and_Graphs as sg

# Other packages
import numpy as np 
import math
import time
import matplotlib.pyplot as plt

In [ ]:
print('hello')

In [2]:
# Qiskit functions
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, transpile
from qiskit.quantum_info import Statevector, Clifford, Pauli, PauliList
from qiskit.circuit import Instruction
from qiskit.circuit.classical import expr
from qiskit_aer import AerSimulator
from qiskit.transpiler.passes import RemoveBarriers
from qiskit.transpiler import CouplingMap

# Other funtions
from typing import Optional
from __future__ import annotations
from IPython.display import Math
from numpy.random import default_rng
from collections import deque, Counter

In [9]:
def outcome_string_to_Pin_step_iii(outcome, n):
    """ """
    N = 2**n
    num_qubits = 2*N-1 + 2*(2*N-n-2)

    x = np.zeros(num_qubits, dtype=bool)
    z = np.array([0]*(N-1) + outcome + [0]*2*(2*N-n-2), dtype=bool)
    P_in = Pauli((z, x))

    return P_in

def precompute_step_iii_corrections(n):
    """ """
    N = 2**n
    
    all_outcomes = grl.all_binary_lists(N)

    C = Clifford(rsp.make_C_NOHE_circuit(n+1 , measure=False)[0])  # note n+1 instead of n
    d = {}

    for outcome in all_outcomes:
        P_in = outcome_string_to_Pin_step_iii(outcome, n)
        P_out = P_in.evolve(C, frame="s")
        d[grl.bits_to_int(outcome)] = P_out

    return d
    

def query_step_iii(qc, n, save_svs=False):
    """ """
    def outcome_string_to_Pin_step_iii(outcome, n):
        """ """
        N = 2**n
        num_qubits = 2*N-1 + 2*(2*N-n-2)
    
        x = np.zeros(num_qubits, dtype=bool)
        z = np.array([0]*(N-1) + outcome + [0]*2*(2*N-n-2), dtype=bool)
        P_in = Pauli((z, x))
    
        return P_in
    
    def precompute_step_iii_corrections(n):
        """ """
        N = 2**n
        
        all_outcomes = grl.all_binary_lists(N)
    
        C = Clifford(rsp.make_C_NOHE_circuit(n+1 , measure=False)[0])  # note n+1 instead of n
        d = {}
    
        for outcome in all_outcomes:
            P_in = outcome_string_to_Pin_step_iii(outcome, n)
            P_out = P_in.evolve(C, frame="s")
            d[grl.bits_to_int(outcome)] = P_out
    
        return d
        
    N = 2**n
    third_wires = qc.metadata[f"C_NOHE_third_wires[{2*N}]"]

    L      = grl.get_qreg(qc, "L")
    K2     = grl.get_qreg(qc, "K2")
    A2     = grl.get_qreg(qc, "A2")
    R_clr  = grl.get_creg(qc, "R_clr")
    I_clr  = grl.get_creg(qc, "I_clr")
    L_clr  = grl.get_creg(qc, "L_clr")
    K2_clr = grl.get_creg(qc, "K2_clr")

    # measure L in X basis
    qc.h(L)
    qc.measure(L, L_clr)

    if save_svs:
        qc.save_statevector(label="meas L in X done", pershot=True)
        
    # precompute pauli corrections
    P_out_dict = precompute_step_iii_corrections(n)

    # rotate q2's in K2 back to Z frame before applying corrections
    for q2 in third_wires:
        qc.h(q2)

    if save_svs:
        qc.save_statevector(label="rotate third wires to Z done", pershot=True)
        
    # apply pauli corrections
    for outcome, P_out in P_out_dict.items():
        with qc.if_test((L_clr, outcome)):
            Zarr, Xarr = P_out.z, P_out.x
            j = 0
            for q in (K2[:] + A2[:]):
                if Zarr[j]:
                    qc.z(q)
                if Xarr[j]:
                    qc.x(q)
                j += 1

    if save_svs:
        qc.save_statevector(label="step iii pauli applied", pershot=True)

    # measure q2's in K2 in X basis
    j = 0
    for q2 in third_wires:
        qc.h(q2)
        qc.measure(q2, K2_clr[j])
        j += 1

    if save_svs:
        qc.save_statevector(label="meas third wires in X done", pershot=True)

    # apply adaptive U_NOHE inversion part
    grl.apply_adaptive_part_U_NOHE_inversion(qc, K2[:] + A2[:], K2_clr[:], n+1, save_svs=save_svs)
    
    if save_svs:
        qc.save_statevector(label="adaptive part done", pershot=True)

    # apply inverse swaps
    grl.apply_inverse_swaps(qc, K2[:])

    if save_svs:
        qc.save_statevector(label="inverse swaps done", pershot=True)
    
    # apply pauli corrections from step i
    for j in range(n):
        Rc  = R_clr[j]
        Ic  = I_clr[j]
        K2q = K2[j]
        with qc.if_test((Rc, 1)):
            qc.z(K2q)
        with qc.if_test((Ic, 1)):
            qc.x(K2q)

    if save_svs:
        qc.save_statevector(label="pauli corr from step i done", pershot=True)

    # H on bus
    bus = K2[n]
    qc.h(bus)

    if save_svs:
        qc.save_statevector(label="h on bus done", pershot=True)

In [7]:
n = 3
N = 2**n
save_svs = False

mem_bits = [0] * N
mem_bits[0] = 1
# mem_bits[1] = 1
# mem_bits[2] = 1
# mem_bits[3] = 1
addr = 0
vec = np.zeros([N])
vec[addr] = 1
qc_addr = qry.initialize_adresses(n, register_name="R", method="Supply vector", vec=vec)
qc_RS = rsp.make_phi_normal(n, save_svs=save_svs)
qc = qc_RS.tensor(qc_addr)
qc.metadata = qc_RS.metadata | qc_addr.metadata

qry.query_step_i(qc, n, save_svs=save_svs)

L = grl.get_qreg(qc, "L")
I_clr = grl.get_creg(qc, "I_clr")

qry.query_step_ii(qc, L, I_clr, mem_bits, save_svs=save_svs)

qry.query_step_iii(qc, n, save_svs=save_svs)

output_clr = ClassicalRegister(n+1, "output_clr")
qc.add_register(output_clr)
K2 = grl.get_qreg(qc, "K2")
qc.measure(K2[:n+1], output_clr[:])


# grl.print_qubit_table(qc)
# qc.draw("mpl", scale=0.7)

KeyboardInterrupt: 

In [4]:
shots=10
t0 = time.time()
# simulate
sim = AerSimulator(method= "statevector")
tqc = transpile(qc, backend=sim, optimization_level=0)
res = sim.run(tqc, shots=shots, memory=True).result()
t1 = time.time()

print("Success:", res.success)
print("Overall status:", res.to_dict().get("status", ""))
print(f"Elapsed time is {(t1-t0)/60} min")

mem    = res.get_memory()
counts = res.get_counts()

grl.print_qubit_table(qc)

for m in mem:
    print(m)

Success: True
Overall status: COMPLETED
Elapsed time is 0.024243311087290446 min
qubit                                    global  register  reg_idx
------------------------------------------------------------------
<Qubit register=(1, "R"), index=0>            0  R               0
<Qubit register=(1, "I"), index=0>            1  I               0
<Qubit register=(1, "D_NOHE"), index=0>       2  D_NOHE          0
<Qubit register=(2, "L"), index=0>            3  L               0
<Qubit register=(2, "L"), index=1>            4  L               1
<Qubit register=(3, "Dprime"), index=0>       5  Dprime          0
<Qubit register=(3, "Dprime"), index=1>       6  Dprime          1
<Qubit register=(3, "Dprime"), index=2>       7  Dprime          2
<Qubit register=(3, "K2"), index=0>           8  K2              0
<Qubit register=(3, "K2"), index=1>           9  K2              1
<Qubit register=(3, "K2"), index=2>          10  K2              2
<Qubit register=(2, "A2"), index=0>          11 

In [17]:
def QRAM_query(qc_addr, qc_RS, n, mem_bits, save_svs=False, meas_output=False):
    """ """
    # combine address and resource states / circuits
    qc = qc_RS.tensor(qc_addr)
    qc.metadata = qc_RS.metadata | qc_addr.metadata

    # step i
    qry.query_step_i(qc, n, save_svs=save_svs)

    # step ii
    L = grl.get_qreg(qc, "L")
    I_clr = grl.get_creg(qc, "I_clr")
    qry.query_step_ii(qc, L, I_clr, mem_bits, save_svs=save_svs)

    # step iii
    qry.query_step_iii(qc, n, save_svs=save_svs)

    if meas_output:
        output_clr = ClassicalRegister(n+1, "output_clr")
        qc.add_register(output_clr)
        K2 = grl.get_qreg(qc, "K2")
        qc.measure(K2[:n+1], output_clr[:])

    return qc

def simulate_circuit(qc, shots=10, method="statevector", print_mem=False):
    """ """
    t0 = time.time()
    # simulate
    sim = AerSimulator(method= method)
    tqc = transpile(qc, backend=sim, optimization_level=0)
    res = sim.run(tqc, shots=shots, memory=True).result()
    t1 = time.time()
    
    print("Success:", res.success)
    print("Overall status:", res.to_dict().get("status", ""))
    print(f"Elapsed time is {(t1-t0)/60} min")
    
    mem    = res.get_memory()
    counts = res.get_counts()

    if print_mem:
        for m in mem:
            print(m)

    return mem, counts

In [3]:
# input variables
n = 3
N = 2**n
save_svs = False

# prepare mem bits
mem_bits = [0] * N
mem_bits[0] = 1
# mem_bits[1] = 1
# mem_bits[2] = 1
# mem_bits[3] = 1

# prepare addr and RS
addr_vec = np.zeros(N, int); addr = 0; addr_vec[addr] = 1 
qc_addr = qry.initialize_adresses(n, register_name="R", method="Supply vector", vec=addr_vec)
qc_RS   = rsp.make_phi_normal(n, save_svs=save_svs)

# run query
qc = qry.QRAM_query(qc_addr, qc_RS, n, mem_bits, save_svs=False, meas_output=True)

KeyboardInterrupt: 

In [19]:
mem, counts = grl.simulate_circuit(qc, shots=20, method="matrix_product_state", print_mem=True)

Success: True
Overall status: COMPLETED
Elapsed time is 1.4014722228050231 min
100 00 11 0110 0011 00 11 0010011011 1011
100 10 00 0111 1101 01 11 0110010001 1001
100 10 01 0100 0000 10 10 1100111101 1011
100 10 01 0101 0100 11 11 0110010011 0110
100 01 01 1100 0001 00 00 1011011111 1001
100 01 10 1001 1101 10 00 1001110101 0011
100 10 10 0100 1111 00 01 0100100110 0010
100 10 11 1001 0010 00 00 1000110001 1000
100 11 11 0011 1011 01 00 0010010101 0101
100 11 01 1000 0101 11 10 0001010111 1000
100 11 10 0001 0101 00 01 0011010110 0111
100 01 01 1010 1111 00 00 0010110111 0000
100 11 10 1100 1100 01 10 1110100000 1101
100 01 11 1010 0111 00 01 1101101101 1111
100 00 00 0011 1111 11 11 1000000101 1110
100 11 00 0011 0100 01 01 0110101101 1000
100 00 11 1110 0111 01 10 1110101111 1111
100 01 10 0111 1010 10 01 1010110100 1001
100 11 01 0110 1001 10 10 1101011100 1111
100 01 10 1011 1001 01 11 1011100001 1100
